In [1]:
import numpy as np
from astropy import units as u
from astropy.coordinates import Angle, SkyCoord, get_constellation
from astroquery.simbad import Simbad
from astroquery.exceptions import RemoteServiceError

import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.6f}'.format)

pd.set_option('display.max_columns', None)  # Ensure all columns are shown
pd.set_option('display.max_colwidth', None)  # Display full content of each field

In [2]:
# Initial setup of Simbad
Simbad.TIMEOUT = 120  # seconds
Simbad.add_votable_fields('otype', 'flux(V)', 'id(NGC)', 'id(M)')

def query_dso_objects(df):
    for index, row in df.iterrows():
        result_table = Simbad.query_object(row['displayName'])
        if result_table is not None:
            # Convert RA / Dec to decimal degree values.
            ra_hour = Angle(result_table['RA'][0], u.hour).value
            dec_deg = Angle(result_table['DEC'][0], u.deg).value
            df.at[index, 'ra'] = ra_hour
            df.at[index, 'dec'] = dec_deg
            
            # Determine the constellation
            coord = SkyCoord(ra=ra_hour * u.hour, dec=dec_deg * u.deg)
            df.at[index, 'constellation'] = get_constellation(coord)
            
            # Set Object
            df.at[index, 'subType'] = result_table['OTYPE'][0]
            #print(f"{row['displayName']}:{df.at[index, 'subType']}")
            
            # Set Flux
            if 'FLUX_V' in result_table.colnames and not np.ma.is_masked(result_table['FLUX_V'][0]):
                df.at[index, 'magnitude'] = float(result_table['FLUX_V'][0])
            else:
                df.at[index, 'magnitude'] = pd.NA
                
            # Extract NGC and IC IDs using the correct column names
            messier_id = result_table['ID_M'][0] if 'ID_M' in result_table.colnames and result_table['ID_M'][0] else None
            ngc_id = result_table['ID_NGC'][0] if 'ID_NGC' in result_table.colnames and result_table['ID_NGC'][0] else None
            
            df.at[index, 'ngcId'] = ngc_id
            
            # Set the objectId based on priority: Messier -> NGC -> IC
            if messier_id:
                df.at[index, 'objectId'] = messier_id.replace(' ', '').lower()
            elif ngc_id:
                df.at[index, 'objectId'] = ngc_id.replace(' ', '').lower()
            else:
                df.at[index, 'objectId'] = pd.NA
        else:
            print(f"No data found for {row['displayName']}")  


In [3]:
messier_object_info = {
    "M1": {"displayName": "Crab Nebula", "subType": "Supernova Remnant", "field": "f/10", "UHC": True, "OIII": True},
    "M2": {"displayName": "M2", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M3": {"displayName": "M3", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M4": {"displayName": "M4", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M5": {"displayName": "M5", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M6": {"displayName": "Butterfly Cluster", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M7": {"displayName": "Ptolemy Cluster", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M8": {"displayName": "Lagoon Nebula", "subType": "Emission Nebula", "field": "f/6.3", "UHC": True, "OIII": False},
    "M9": {"displayName": "M9", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M10": {"displayName": "M10", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M11": {"displayName": "Wild Duck Cluster", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M12": {"displayName": "M12", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M13": {"displayName": "Hercules Cluster", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M14": {"displayName": "M14", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M15": {"displayName": "M15", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M16": {"displayName": "Eagle Nebula", "subType": "Emission Nebula", "field": "f/6.3", "UHC": True, "OIII": False},
    "M17": {"displayName": "Omega Nebula", "subType": "Emission Nebula", "field": "f/6.3", "UHC": True, "OIII": False},
    "M18": {"displayName": "M18", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M19": {"displayName": "M19", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M20": {"displayName": "Trifid Nebula", "subType": "Emission/Reflection Nebula", "field": "f/6.3", "UHC": True, "OIII": False},
    "M21": {"displayName": "M21", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M22": {"displayName": "M22", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M23": {"displayName": "M23", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M24": {"displayName": "Sagittarius Star Cloud", "subType": "Star Cloud", "field": "f/6.3", "UHC": False, "OIII": False},
    "M25": {"displayName": "M25", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M26": {"displayName": "M26", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M27": {"displayName": "Dumbbell Nebula", "subType": "Planetary Nebula", "field": "f/10", "UHC": True, "OIII": True},
    "M28": {"displayName": "M28", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M29": {"displayName": "M29", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M30": {"displayName": "M30", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M31": {"displayName": "Andromeda Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M32": {"displayName": "M32", "subType": "Dwarf Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M33": {"displayName": "Triangulum Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M34": {"displayName": "M34", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M35": {"displayName": "M35", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M36": {"displayName": "M36", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M37": {"displayName": "M37", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M38": {"displayName": "M38", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M39": {"displayName": "M39", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M40": {"displayName": "Winnecke 4", "subType": "Double Star", "field": "f/10", "UHC": False, "OIII": False},
    "M41": {"displayName": "M41", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M42": {"displayName": "Orion Nebula", "subType": "Emission Nebula", "field": "f/6.3", "UHC": True, "OIII": True},
    "M43": {"displayName": "De Mairan's Nebula", "subType": "Emission Nebula", "field": "f/6.3", "UHC": True, "OIII": False},
    "M44": {"displayName": "Beehive Cluster", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M45": {"displayName": "Pleiades", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M46": {"displayName": "M46", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M47": {"displayName": "M47", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M48": {"displayName": "M48", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M49": {"displayName": "M49", "subType": "Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M50": {"displayName": "M50", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M51": {"displayName": "Whirlpool Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M52": {"displayName": "M52", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M53": {"displayName": "M53", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M54": {"displayName": "M54", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M55": {"displayName": "M55", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M56": {"displayName": "M56", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M57": {"displayName": "Ring Nebula", "subType": "Planetary Nebula", "field": "f/10", "UHC": True, "OIII": True},
    "M58": {"displayName": "M58", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M59": {"displayName": "M59", "subType": "Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M60": {"displayName": "M60", "subType": "Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M61": {"displayName": "M61", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M62": {"displayName": "M62", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M63": {"displayName": "Sunflower Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M64": {"displayName": "Black Eye Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M65": {"displayName": "Leo Triplet", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M66": {"displayName": "Leo Triplet", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M67": {"displayName": "M67", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M68": {"displayName": "M68", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M69": {"displayName": "M69", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M70": {"displayName": "M70", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M71": {"displayName": "M71", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M72": {"displayName": "M72", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M73": {"displayName": "M73", "subType": "Asterism", "field": "f/10", "UHC": False, "OIII": False},
    "M74": {"displayName": "Phantom Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M75": {"displayName": "M75", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M76": {"displayName": "Little Dumbbell Nebula", "subType": "Planetary Nebula", "field": "f/10", "UHC": True, "OIII": True},
    "M77": {"displayName": "M77", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M78": {"displayName": "M78", "subType": "Reflection Nebula", "field": "f/6.3", "UHC": False, "OIII": False},
    "M79": {"displayName": "M79", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M80": {"displayName": "M80", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M81": {"displayName": "Bode's Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M82": {"displayName": "Cigar Galaxy", "subType": "Starburst Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M83": {"displayName": "Southern Pinwheel Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M84": {"displayName": "M84", "subType": "Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M85": {"displayName": "M85", "subType": "Lenticular Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M86": {"displayName": "M86", "subType": "Lenticular Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M87": {"displayName": "Virgo A", "subType": "Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M88": {"displayName": "M88", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M89": {"displayName": "M89", "subType": "Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M90": {"displayName": "M90", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M91": {"displayName": "M91", "subType": "Barred Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M92": {"displayName": "M92", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M93": {"displayName": "M93", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M94": {"displayName": "M94", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M95": {"displayName": "M95", "subType": "Barred Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M96": {"displayName": "M96", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M97": {"displayName": "Owl Nebula", "subType": "Planetary Nebula", "field": "f/10", "UHC": True, "OIII": True},
    "M98": {"displayName": "M98", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M99": {"displayName": "Pinwheel Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M100": {"displayName": "M100", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M101": {"displayName": "M101", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M102": {"displayName": "Spindle Galaxy", "subType": "Lenticular Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M103": {"displayName": "M103", "subType": "Open Cluster", "field": "f/6.3", "UHC": False, "OIII": False},
    "M104": {"displayName": "Sombrero Galaxy", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M105": {"displayName": "M105", "subType": "Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M106": {"displayName": "M106", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M107": {"displayName": "M107", "subType": "Globular Cluster", "field": "f/10", "UHC": False, "OIII": False},
    "M108": {"displayName": "M108", "subType": "Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M109": {"displayName": "M109", "subType": "Barred Spiral Galaxy", "field": "f/10", "UHC": False, "OIII": False},
    "M110": {"displayName": "M110", "subType": "Dwarf Elliptical Galaxy", "field": "f/10", "UHC": False, "OIII": False},
}

def create_messier_catalog():
    messier_objects = [f'M{i}' for i in range(1, 111)]
    df = pd.DataFrame(messier_objects, columns=['displayName']) 
    df['objectId'] = df['displayName'].apply(lambda x: 'm' + x[1:])

    return df

def apply_messier_info(row):
    object_id_upper = row['objectId'].upper()  # Convert objectId to uppercase to match keys in object_info
    if object_id_upper in messier_object_info:
        info = messier_object_info[object_id_upper]
        row['displayName'] = info['displayName']
        row['subType'] = info['subType']
        row['field'] = info['field']
        row['UHC'] = info['UHC']
        row['OIII'] = info['OIII']
    else:
        # Default values if objectId not found in object_info
        row['displayName'] = row['displayName']  # Keep existing displayName
        row['subType'] = 'Unknown'
        row['field'] = 'f/10'
        row['UHC'] = False
        row['OIII'] = False
    return row

subtype_to_type = {
    'HIIG': 'galaxy',
    'Supernova Remnant': 'nebula',
    'Globular Cluster': 'cluster',
    'Open Cluster': 'cluster',
    'Emission Nebula': 'nebula',
    'Emission/Reflection Nebula': 'nebula',
    'Star Cloud': 'nebula',  # Star clouds are often part of nebulous regions
    'Planetary Nebula': 'nebula',
    'Spiral Galaxy': 'galaxy',
    'Dwarf Elliptical Galaxy': 'galaxy',
    'Double Star': 'star',
    'Elliptical Galaxy': 'galaxy',
    'Asterism': 'star',
    'Reflection Nebula': 'nebula',
    'Starburst Galaxy': 'galaxy',
    'Lenticular Galaxy': 'galaxy',
    'Barred Spiral Galaxy': 'galaxy',
    'OpenCluster': 'cluster',
    'ISM': 'nebula',           # Interstellar Medium, often categorized under nebulae
    'HIIReg': 'nebula',        # HII Regions are part of emission nebulae
    'PlanetaryNeb': 'nebula',  # Planetary Nebula
    'DarkNeb': 'nebula',       # Dark Nebula
    'Seyfert1': 'galaxy',      # Seyfert galaxies are active galaxies
    'Cluster*': 'cluster',     # Star clusters
    'Compact_Gr_G': 'galaxy',  # Compact Group of Galaxies
    'LINER': 'galaxy'          # Galaxy Group (Caldwell 30)
}

messier_recommended = ['m1', 'm7', 'm8', 'm11', 'm13', 'm16', 'm17', 'm20', 'm24',
                       'm27', 'm31', 'm33', 'm42', 'm44', 'm45', 'm51', 'm57', 'm58', 
                       'm59', 'm61', 'm63', 'm64', 'm66', 'm74', 'm77', 'm78', 'm81', 
                       'm82', 'm83', 'm88', 'm94', 'm95', 'm96', 'm97', 'm98', 'm100', 
                       'm101', 'm102', 'm104', 'm106', 'm108', 'm109', 'm110']  

column_order = [
    'displayName', 'objectId', 'ra', 'dec', 'constellation', 'type', 
    'subType', 'magnitude', 'ngcId', 'recommended', 'field', 'UHC', 'OIII'
]

In [4]:
df_messier = create_messier_catalog()
query_dso_objects(df_messier)
df_messier = df_messier.apply(apply_messier_info, axis=1)
df_messier['type'] = df_messier['subType'].map(subtype_to_type)
df_messier['recommended'] = False
df_messier.loc[df_messier['objectId'].isin(messier_recommended), 'recommended'] = True
df_messier = df_messier[column_order]

In [5]:
def create_dsos():
    df = pd.DataFrame({
        'displayName': pd.Series(dtype='str'),
        'objectId': pd.Series(dtype='object'),
        'ra': pd.Series(dtype='object'),
        'dec': pd.Series(dtype='object'),
        'constellation': pd.Series(dtype='object'),
        'type': pd.Series(dtype='object'),
        'subType': pd.Series(dtype='object'),
        'magnitude': pd.Series(dtype='object'),
        'ngcId': pd.Series(dtype='object'),
        'recommended': pd.Series(dtype='bool'),
        'field': pd.Series(dtype='object'),
        'UHC': pd.Series(dtype='bool'),
        'OIII': pd.Series(dtype='bool')
    })
    
    return df


def add_dso_object(df, displayName, field='f/10', UHC=False, OIII=False):
    # Create a new row DataFrame with specified values and defaults to pd.NA
    new_row = pd.DataFrame({
        'displayName': [displayName],
        'objectId': [pd.NA],
        'ra': [pd.NA],
        'dec': [pd.NA],
        'constellation': [pd.NA],
        'type': [pd.NA],
        'subType': [pd.NA],
        'magnitude': [pd.NA],
        'ngcId': [pd.NA],
        'recommended': [bool(True)],
        'field': [field],
        'UHC': [bool(UHC)],  # Explicitly cast to bool
        'OIII': [bool(OIII)]  # Explicitly cast to bool
    })

    # Concatenate the new row to the DataFrame
    df = pd.concat([df, new_row], ignore_index=True)
    
    return df

def assign_missing_objectIds (df):
    df.loc[df['displayName'] == "Rosette Nebula", 'objectId'] = 'ldn1625'
    df.loc[df['displayName'] == "Horsehead Nebula", 'objectId'] = 'ic434'
    df.loc[df['displayName'] == 'Wizard Nebula', 'objectId'] = 'ngc7380'
    df.loc[df['displayName'] == "Stephan's Quintet", 'objectId'] = 'ngc7331'
    df.loc[df['displayName'] == "Cocoon Nebula", 'objectId'] = 'ic5146'
    df.loc[df['displayName'] == "Cocoon Nebula", 'subType'] = 'Emission/Reflection Nebula'
    df.loc[df['displayName'] == "Sh 2-101", 'objectId'] = 'Sh2-101'
    df.loc[df['displayName'] == "Sh 2-101", 'displayName'] = 'Sh2-101 (Tulip Nebula)'
    df.loc[df['displayName'] == "NGC 7331", 'displayName'] = 'Caldwell 30 (Galaxy Group)'
    df.loc[df['displayName'] == "NGC 7662", 'displayName'] = 'Blue Snowball'
    df.loc[df['displayName'] == "NGC 6826", 'displayName'] = 'Blinking Planetary'
    df.loc[df['displayName'] == "NGC 2392", 'displayName'] = 'Eskimo Nebula'
    df.loc[df['displayName'] == "NGC 7009", 'displayName'] = 'Saturn Nebula'
    df.loc[df['displayName'] == "NGC 281", 'displayName'] = 'Pac Man Nebula'
    
    df.loc[df['displayName'] == "NGC 1499", 'subType'] = 'Emission Nebula'  
    df.loc[df['displayName'] == "NGC 1499", 'displayName'] = 'California Nebula'
    

    df.loc[df['displayName'] == "NGC 2264", 'subType'] = 'Emission Nebula'     
    df.loc[df['displayName'] == "NGC 2264", 'displayName'] = 'Cone Nebula'
    
    
    df.loc[df['displayName'] == "IC 1805", 'objectId'] = 'ic1805'
    df.loc[df['displayName'] == "IC 1805", 'subType'] = 'Emission Nebula' 
    df.loc[df['displayName'] == "IC 1805", 'displayName'] = 'Heart Nebula'
    
    df.loc[df['displayName'] == "IC 1848", 'objectId'] = 'ic1848'
    df.loc[df['displayName'] == "IC 1848", 'subType'] = 'Emission Nebula' 
    df.loc[df['displayName'] == "IC 1848", 'displayName'] = 'Soul Nebula'
    
    df.loc[df['displayName'] == "NGC 2467", 'subType'] = 'Emission Nebula'    
    df.loc[df['displayName'] == "NGC 2467", 'displayName'] = 'Skull and Crossbones Nebula'    
    
    df.loc[df['displayName'] == "NGC1977", 'subType'] = 'Emission Nebula'    
    df.loc[df['displayName'] == "NGC1977", 'displayName'] = 'Running Man Nebula'    

In [6]:
df_dso = create_dsos()

df_dso = add_dso_object(df_dso, 'NGC 869', field='f/6.3', UHC=False, OIII=False)
df_dso = add_dso_object(df_dso, 'Veil Nebula', field='f/6.3', UHC=True, OIII=True)
df_dso = add_dso_object(df_dso, 'Rosette Nebula', field='f/6.3', UHC=True, OIII=False)
df_dso = add_dso_object(df_dso, 'Helix Nebula', field='f/10', UHC=True, OIII=True)
df_dso = add_dso_object(df_dso, 'Tau CMa', field='f/10', UHC=False, OIII=False)
df_dso = add_dso_object(df_dso, 'Horsehead Nebula', field='f/10', UHC=True, OIII=False)
df_dso = add_dso_object(df_dso, 'Wizard Nebula', field='f/6.3', UHC=True, OIII=False)
df_dso = add_dso_object(df_dso, 'North America Nebula', field='f/6.3', UHC=True, OIII=False)
df_dso = add_dso_object(df_dso, 'Crescent Nebula', field='f/6.3', UHC=True, OIII=True)
df_dso = add_dso_object(df_dso, "Stephan's Quintet", field='f/10', UHC=False, OIII=False)
df_dso = add_dso_object(df_dso, "Cocoon Nebula", field='f/6.3', UHC=True, OIII=False)
df_dso = add_dso_object(df_dso, "NGC1977", field='f/6.3', UHC=False, OIII=False)
#df_dso = add_dso_object(df_dso, "Sh 2-101", field='f/6.3', UHC=True, OIII=False)
#df_dso = add_dso_object(df_dso, "NGC 7331", field='f/10', UHC=False, OIII=False)
#df_dso = add_dso_object(df_dso, "NGC 7662", field='f/10', UHC=True, OIII=True)
#df_dso = add_dso_object(df_dso, "NGC 6826", field='f/10', UHC=True, OIII=True)
#df_dso = add_dso_object(df_dso, "NGC 2392", field='f/10', UHC=True, OIII=True)
#df_dso = add_dso_object(df_dso, "NGC 7009", field='f/10', UHC=True, OIII=True)
#df_dso = add_dso_object(df_dso, "NGC 281", field='f/10', UHC=True, OIII=True)
df_dso = add_dso_object(df_dso, "NGC 1499", field='f/6.3', UHC=True, OIII=True)
df_dso = add_dso_object(df_dso, "NGC 2264", field='f/6.3', UHC=True, OIII=True)
df_dso = add_dso_object(df_dso, "IC 1805", field='f/6.3', UHC=True, OIII=True)
df_dso = add_dso_object(df_dso, "IC 1848", field='f/6.3', UHC=True, OIII=True)
df_dso = add_dso_object(df_dso, "NGC 2467", field='f/6.3', UHC=True, OIII=True)

query_dso_objects(df_dso)
assign_missing_objectIds(df_dso)
df_dso['type'] = df_dso['subType'].map(subtype_to_type)

In [7]:
df = pd.concat([df_dso, df_messier], ignore_index=True)
df['id'] = 0
df.to_json('dso.json', orient='records', lines=False)

In [8]:
df_dso[0:200]

,displayName,objectId,ra,dec,constellation,type,subType,magnitude,ngcId,recommended,field,UHC,OIII
0,NGC 869,ngc869,2.316056,57.133889,Perseus,cluster,OpenCluster,<NA>,NGC 869,True,f/6.3,False,False
1,Veil Nebula,ngc6960,20.760556,30.708333,Cygnus,nebula,ISM,<NA>,NGC 6960,True,f/6.3,True,True
2,Rosette Nebula,ldn1625,6.540767,4.793639,Monoceros,nebula,HIIReg,<NA>,None,True,f/6.3,True,False
3,Helix Nebula,ngc7293,22.494040,-20.837152,Aquarius,nebula,PlanetaryNeb,13.524000,NGC 7293,True,f/10,True,True
4,Tau CMa,ngc2362,7.311389,-24.953889,Canis Major,cluster,OpenCluster,<NA>,NGC 2362,True,f/10,False,False
5,Horsehead Nebula,ic434,5.683056,-2.458333,Orion,nebula,DarkNeb,<NA>,None,True,f/10,True,False
6,Wizard Nebula,ngc7380,22.792222,58.048333,Cepheus,nebula,HIIReg,<NA>,None,True,f/6.3,True,False
7,North America Nebula,ngc7000,20.979722,44.330000,Cygnus,cluster,Cluster*,<NA>,NGC 7000,True,f/6.3,True,False
8,Crescent Nebula,ngc6888,20.201944,38.355000,Cygnus,nebula,ISM,<NA>,NGC 6888,True,f/6.3,True,True
9,Stephan's Quintet,ngc7331,22.599306,33.960000,Pegasus,galaxy,Compact_Gr_G,<NA>,None,True,f/10,False,False
